A faire : 
- parcourir toutes les dates
- cliquer sur un jour 
- cliquer sur un tournoi
- aller dans l'onglet "grille" (s'il y en a) et lire les informations : 
- mettre dans un tableau, pour chaque joueur le nom du tournoi, le classement du joueur dans le tournoi, contre qui il a joué (avec le score) 

#indication : =/- : gagné, nul, perdu , 4 : chiffre de l’adversaire, B/N : blancs ou noirs 

In [69]:
import requests

out = requests.get("http://echecs.asso.fr/FicheTournoi.aspx?Ref=55579")
out.headers
print(out.content.decode())



<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" >
<head id="ctl00_MasterHeader"><meta http-equiv="content-type" content="application/xhtml+xml; charset=utf-8" /><meta name="keywords" /><meta name="description" /><title>
	F&eacute;d&eacute;ration Fran&ccedil;aise des &Eacute;checs
</title><link type="text/css" rel="stylesheet" href="http://fonts.googleapis.com/css?family=Source+Sans+Pro:200,300,400,600" /><link href="http://fonts.googleapis.com/css?family=Oswald:300,400,700" rel="stylesheet" type="text/css" /><link type="text/css" rel="stylesheet" href="ffe.css?version=6" /><link type="text/css" rel="stylesheet" href="old.css?version=1" /><link rel="shortcut icon" type="image/x-icon" href="images/ffe16.ico" />
  <script>
    (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
    (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o)

In [70]:
import requests
from bs4 import BeautifulSoup

link = "http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/55579/55579&Action=Ga"
soup = BeautifulSoup(requests.get(link).text, 'html.parser')

#print(soup)

for x in soup.find_all('div'):
    if x.has_attr("class") and x['class'][0] == "papi_joueur_box":
        print(x.b.string)
        
adversaires = []
elo = []
perf = []
couleur = []
résultat = []

for x in soup.find_all('div'):
    if x.has_attr("class") and x.has_attr("align") and x['align'] == "center":
        joueur = []
        for k,y in enumerate(x.find_all('td')):
            print(y.string)
            if (k-16)%13==0:
                joueur.append(y.string)
            if (k-17)%13==0:
                z=str(y.string).split("\xa0")
                elo.append(z)
            if (k-23)%13==0:
                perf.append(y.string)
            if (k-12)%13==0:
                couleur.append(y.string)
            if k%13==0:
                résultat.append(y.string)
        adversaires.append(joueur) 

print(adversaires)
print(couleur)
print(résultat)
print(elo)
print(perf)

dico_adversaires = dict()
for x in adversaires :
    dico_adversaires[x[0]]=[x[1]]
    for k in range(2, len(x)):
        dico_adversaires[x[0]].append(x[k])
                

MARCHAL Philippe
PERCHE Ludovic
MARCHAL Jean-Luc
FOUGERAT Thierry
KIEHN Laurent
DESPRET Daniel
ADELIN Jonathan
BOMBARD Ilan
PRIEUR Axel
PLASSART Jean-Luc
DUPRE Elena
BEAUFILS Martial
MARCHAL Sacha
LENROUILLY Arthur
MARCHAL Jean-Marie
GOUY PITON Sylvain
FRADET Jacques
 
1
 
MARCHAL Philippe
1452 F
SenM
FRA
NAQ
3
4
2071
1
N
1
7
 
ADELIN Jonathan
1199 E
SenM
FRA
NAQ
2
3
1443
2
B
>
11
 
DUPRE Elena
1334 F
SepF
FRA
NAQ
1
4½
1932
3
B
1
6
 
DESPRET Daniel
1367 F
SepM
FRA
NAQ
2
3½
1451
 
2
 
PERCHE Ludovic
1494 F
SepM
FRA
NAQ
3
4
2040
1
B
1
10
 
PLASSART Jean-Luc
1223 F
VetM
FRA
NAQ
1
5½
1257
2
N
1
4
 
FOUGERAT Thierry
1416 F
SepM
FRA
NAQ
2
4½
1560
3
B
1
5
 
KIEHN Laurent
1450 F
SenM
FRA
NAQ
2
4½
1465
 
3
 
MARCHAL Jean-Luc
1382 F
SenM
FRA
NAQ
2½
2½
1624
1
B
1
16
 
GOUY PITON Sylvain
1199 E
BenM
FRA
NAQ
0
4½
724
2
N
½
9
 
PRIEUR Axel
1563 F
SenM
FRA
NAQ
1½
4½
1351
3
B
1
10
 
PLASSART Jean-Luc
1223 F
VetM
FRA
NAQ
1
5½
1257
 
4
 
FOUGERAT Thierry
1416 F
SepM
FRA
NAQ
2
4½
1560
1
N
1
17
 
FRADET J

In [71]:
base = dict()

number = 55579

#checker si la page est américaine 
def american(number):
    link = f"http://echecs.asso.fr/Resultats.aspx?URL=Tournois/Id/{number}/{number}&Action=Ga"
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    for x in soup.find_all('span'):
        if x.string == "Désolé, le fichier n'existe pas...":
            print(x.string)
            return False
        else :
            print("c'est ok")
            return True 


#pour avoir la date, le nom, la cadence, et les rondes du tournoi
def infos_tournoi(number):
    infos = dict()
    link_infos = f"http://echecs.asso.fr/FicheTournoi.aspx?Ref={number}"
    soup_infos = BeautifulSoup(requests.get(link_infos).text, 'html.parser')
    for x in soup_infos.find_all('span'):
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelDates":
            infos["date"] = x.string   
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelNom":
            infos["nom"] = x.string  
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelNbrRondes":
            infos["nb_rondes"] = x.string
        if x.has_attr("id") and x['id'] == "ctl00_ContentPlaceHolderMain_LabelCadence":
            infos["cadence"] = x.string 
    return infos



#dans la boucle, si la date c'est demain, on arrête la boucle 

from datetime import datetime

months = {"janvier":1, "février":2, "mars":3, "avril":4,
       "mai":5, "juin":6, "juillet":7, "août":8,
       "septembre":9, "octobre":10, "novembre":11, "décembre":12}

def stop_boucle(date): #fonction qui renvoie true si on doit arrêter la boucle
    liste_date = date.strip('-').split(' ')
    for k in range(len(liste_date)):
        if k%5==1:
            print(liste_date[k])
            jour = liste_date[k]
        if k%5==2:
            print(k)
            mois = months[liste_date[k]]
        if k%5==3:
            année = liste_date[k]
    jour_now, mois_now, année_now = datetime.today().day, datetime.today().month, datetime.today().year
    if int(année)>int(année_now):
        return True
    elif int(année)==int(année_now):
        if int(mois)>int(mois_now):
            return True
        elif int(mois)==int(mois_now):
            if int(jour)>int(jour_now):
                return True
    return False
        
print(stop_boucle('mercredi 23 juin 2022 - samedi 8 juillet 2022'))
    

adversaires = []


for x in soup.find_all('div'):
    if x.has_attr("class") and x.has_attr("align") and x['align'] == "center":
        joueur = []
        for k,y in enumerate(x.find_all('td')):
            if (k-16)%13==0:
                joueur.append(y.string)
        adversaires.append(joueur) 

for x in adversaires :
    if x[0] not in base.keys():
        base[x[0]]=dict()
        adversaire = dict()
    for k in range(1, len(x)):
        if x[k] not in base[x[0]].keys():
            adversaire[x[k]] = []
        tournoi = dict()
        tournoi["nom du tournoi"]= nom
        adversaire[x[k]].append(tournoi)  
    base[x[0]]=adversaire
    

23
2
8
7
True
